In [1]:
%reload_ext autoreload
%autoreload 2

In [40]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import GRAY_DASH, load_pkl, save_latex, mini_bar

data: DiffResult = load_pkl("src")

KINDS = [
    DepKind.FUNC,
    # DepKind.LSM,
    DepKind.STRUCT,
    DepKind.TRACEPOINT,
]

ISSUES = {
    IssueEnum.OLD: r"\ \ \ # \ \ ",
    IssueEnum.ADD: r"$+$%",
    IssueEnum.REMOVE: r"$-$%",
    IssueEnum.CHANGE: r"$\Delta$%",
}

COLORS = {
    IssueEnum.ADD: "add",
    IssueEnum.REMOVE: "remove",
    IssueEnum.CHANGE: "change",
}

GROUPS = {
    VersionGroup.LTS: "LTS (2 yr)",
    VersionGroup.REGULAR: "Regular Releases (6 mo)",
}


results = {}


def format_val(kind, val) -> str:
    if kind == DepKind.FUNC:
        return f"{val / 1000:.0f}k"
    elif kind == DepKind.STRUCT:
        return f"{val / 1000:.1f}k"
    return str(val)


max_percent = {
    (kind, issue): max(
        pair_result.data[kind].issues[issue]
        / pair_result.data[kind].issues[IssueEnum.OLD]
        for group, group_result in data.iter_groups()
        for pair, pair_result in group_result.iter_pairs()
    )
    for kind in KINDS
    for issue in ISSUES
}

for group, group_result in data.iter_groups():
    if group not in GROUPS:
        continue

    for pair, pair_result in group_result.iter_pairs():
        result = {}
        for kind, kind_result in pair_result.iter_kinds():
            if kind not in KINDS:
                continue
            old = kind_result.issues[IssueEnum.OLD]
            for issue, count in kind_result.iter_issues():
                if issue not in ISSUES:
                    continue

                if issue == IssueEnum.OLD:
                    text = format_val(kind, count)
                else:
                    if count == 0:
                        text = GRAY_DASH
                    else:
                        text = mini_bar(
                            text=f"{count / old * 100:.0f}",
                            percent=count / old / max_percent[(kind, issue)],
                            total_width=0.55,
                            color=COLORS[issue],
                            bg_color="white",
                        )
                        # text = f"{count / old * 100:.0f}"
                    text = r"\multirow{2}{*}{" + text + "}"
                result[(kind, ISSUES[issue])] = text

        v1 = group.to_str(pair.v1)
        v2 = group.to_str(pair.v2)

        group_name = GROUPS[group]
        results[(group_name, v1)] = result
        if pair.v2 == list(group)[-1]:
            result = {k: "" for k in result.keys()}
            for kind, key in result.keys():
                if key == ISSUES[IssueEnum.OLD]:
                    count = pair_result.data[kind].issues[IssueEnum.NEW]
                    result[(kind, key)] = format_val(kind, count)
            results[(group_name, v2)] = result


df = pd.DataFrame(results).T

latex = df.to_latex(multicolumn_format="c|", column_format=r"cc|rrrr|rrrr|rrrr")
save_latex(latex, "src")

[ utils_pickle.py:18 ] INFO: Loding src from /Users/szhong/Code/DepSurf/output/src.pkl
[  utils_latex.py:105] INFO: Saved src to /Users/szhong/Code/DepSurf/paper/tabs/src.tex


In [42]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import GRAY_DASH, load_pkl, save_latex, mini_bar

data: DiffResult = load_pkl("src")

KINDS = {
    DepKind.FUNC: DepKind.FUNC,
    DepKind.STRUCT: DepKind.STRUCT,
    DepKind.TRACEPOINT: "Tracept",
}

GROUPS = {VersionGroup.LTS}

ISSUES_IGNORE = [
    IssueEnum.ADD,
    IssueEnum.REMOVE,
    IssueEnum.OLD,
    IssueEnum.NEW,
]


results = {}
for group, group_result in data.iter_groups():
    if group not in GROUPS:
        continue

    for pair, pair_result in group_result.iter_pairs():
        result = {}

        for kind, kind_result in pair_result.iter_kinds():
            if kind not in KINDS:
                continue
            change = kind_result.issues[IssueEnum.CHANGE]
            for issue, count in kind_result.iter_issues():
                if issue in ISSUES_IGNORE:
                    continue
                if count == 0:
                    continue
                if kind == DepKind.TRACEPOINT:
                    if issue not in (
                        IssueEnum.CHANGE,
                        IssueEnum.TRACE_EVENT_CHANGE,
                        IssueEnum.TRACE_FUNC_CHANGE,
                    ):
                        continue

                if issue == IssueEnum.CHANGE:
                    issue = "No. changed"
                    text = f"{count / 1000:.1f}k" if count > 1000 else str(count)
                else:
                    issue = f" - {issue}"
                    text = mini_bar(
                        text=f"{count / change * 100:.0f}%",
                        percent=count / change,
                        total_width=0.7,
                        color="change",
                        bg_color="lightgray!20",
                    )

                result[(KINDS[kind], issue)] = text

        v1 = group.to_str(pair.v1)
        v2 = group.to_str(pair.v2)

        def header(v):
            return ("Linux Kernel Version", v)

        if pair.v1 == list(group)[0]:
            results[header(v1)] = {k: "" for k in result.keys()}
        results[header(v2)] = result
        # results[("Linux Kernel Version", v1, v2)] = result

df = pd.DataFrame(results)
df = df.fillna(GRAY_DASH)
# df.columns.names = ["", r"\multicolumn{1}{r|}{From}", r"\multicolumn{1}{r|}{To}"]
df.iloc[:, -1] = df.iloc[:, -1].apply(lambda x: x + " &")
latex = df.to_latex(multicolumn_format="c", column_format="cl|rrrrrr")
latex = latex.replace(
    r"\multicolumn{5}{c}{Linux Kernel Version}",
    r"\multicolumn{6}{c}{Linux Kernel Version}",
)
latex = latex.replace(
    "4.4 & 4.15 & 5.4 & 5.15 & 6.8",
    r"\multicolumn{6}{c}{\quad 4.4 \qquad 4.15 \qquad 5.4 \qquad 5.15 \qquad 6.8 \quad}",
)
save_latex(latex, "breakdown")

[ utils_pickle.py:18 ] INFO: Loding src from /Users/szhong/Code/DepSurf/output/src.pkl
[  utils_latex.py:105] INFO: Saved breakdown to /Users/szhong/Code/DepSurf/paper/tabs/breakdown.tex
